# Workflow Basics

<table style="width:90%">
    <tr><td style="text-align:left;"><a href="./3 - Configurations.ipynb">Previous (Configurations)</a></td><td style="text-align:right;"><a href="./5 - Debugging.ipynb">Next (Debugging)</a></td></tr>
</table>


## Sequential Workflows

Just running tasks one after another, you can even pass AppFutures from task to task

In [1]:
import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config
from parsl.data_provider.files import File

parsl.load(config)

# App that generates a random number
@python_app
def generate(limit):
      from random import randint
      return randint(1,limit)

# App that writes a variable to a file
@bash_app
def save(variable, outputs=[]):
      return 'echo %s &> %s' % (variable, outputs[0])

# Generate a random number between 1 and 10
random = generate(10)
print('Random number: %s' % random.result())

# Save the random number to a file
saved = save(random, outputs=[File(os.path.join(os.getcwd(), 'sequential-output.txt'))])

# Print the output file
with open(saved.outputs[0].result(), 'r') as f:
      print('File contents: %s' % f.read())

## Parallel Workflows

These can be genreated by loops

In [2]:
# App that generates a random number after a delay
@python_app
def generate(limit,delay):
    from random import randint
    import time
    time.sleep(delay)
    return randint(1,limit)

# Generate 5 random numbers between 1 and 10
rand_nums = []
for i in range(5):
    rand_nums.append(generate(10,i))

# Wait for all apps to finish and collect the results
outputs = [i.result() for i in rand_nums]

# Print results
print(outputs)

and by passing data between Apps.

In [3]:
# App that generates a semi-random number between 0 and 32,767
@bash_app
def generate(outputs=[]):
    return "echo $(( RANDOM )) &> {}".format(outputs[0])

# App that concatenates input files into a single output file
@bash_app
def concat(inputs=[], outputs=[]):
    return "cat {0} > {1}".format(" ".join([i.filepath for i in inputs]), outputs[0])

# App that calculates the sum of values in a list of input files
@python_app
def total(inputs=[]):
    total = 0
    with open(inputs[0], 'r') as f:
        for l in f:
            total += int(l)
    return total

# Create 5 files with semi-random numbers in parallel
output_files = []
for i in range (5):
     output_files.append(generate(outputs=[File(os.path.join(os.getcwd(), 'random-{}.txt'.format(i)))]))

# Concatenate the files into a single file
cc = concat(inputs=[i.outputs[0] for i in output_files], 
            outputs=[File(os.path.join(os.getcwd(), 'all.txt'))])

# Calculate the sum of the random numbers
total = total(inputs=[cc.outputs[0]])
print (total.result())

## Dynamic Workflows

These workflows are very adaptable and best used when some stages have an unknown number of steps. We use the `join_app` which is used to define sub-workflows. Unlike the other Apps the `join_app` returns an AppFuture and waits for the future to complete before completing. Its result value will be that of the future.

In [4]:
from parsl.app.app import join_app

@python_app
def add(*args):
    """Add all of the arguments together. If no arguments, then
    zero is returned (the neutral element of +)
    """
    accumulator = 0
    for v in args:
        accumulator += v
    return accumulator


@join_app
def fibonacci(n):
    if n == 0:
        return add()
    elif n == 1:
        return add(1)
    else:
        return add(fibonacci(n - 1), fibonacci(n - 2))
    
print(fibonacci(10).result())

## Exercise

We are going to use the Monte Carlo method to calculate $\pi$. This is done by dropping points in a square and using the ratio that are placed within the circle.

One example is calculating $\pi$ by randomly placing points in a box and using the ratio that are placed inside the circle.

Specifically, if a circle with radius $r$ is inscribed inside a square with side length $2r$, the area of the circle is $\pi r^2$ and the area of the square is $(2r)^2$.

Thus, if $N$ uniformly-distributed random points are dropped within the square, approximately $N\pi/4$ will be inside the circle.

Each call to the function `pi()` is executed independently and in parallel. The  `avg_three()` app is used to compute the average of the futures that were returned from the `pi()` calls.

The dependency chain looks like this:

```
App Calls    pi()  pi()   pi()
              \     |     /
Futures        a    b    c
                \   |   /
App Call        avg_three()
                    |
Future            avg_pi
```

The function that calculates $\pi$ is given below. Your task is:
* Turn this function into an App
* Write an App called avg_three() which takes three AppFutures as arguments and returns the average
* Each call to pi() should be large enough to well sample the box (~10^6 points), feel free to play around with this value to see how many it takes to within 5%, 1%, or 0.1%

In [5]:
# App that estimates pi by placing points in a box
def pi(num_points):
    from random import random
    
    inside = 0   
    for i in range(num_points):
        x, y = random(), random()  # Drop a random point in the box.
        if x**2 + y**2 < 1:        # Count points within the circle.
            inside += 1
    
    return (inside*4 / num_points)



# Print the results
print(f"Average: {pi(10**6):.5f}")

In [6]:
@python_app
def pi(num_points):
    from random import random
    inside = 0   
    for i in range(num_points):
        x, y = random(), random()
        if x**2 + y**2 < 1:      
            inside += 1
    return (inside*4 / num_points)

@python_app
def avg_three(a, b, c):
    return (a + b + c) / 3


a, b, c = pi(10**6), pi(10**6), pi(10**6)


avg_pi  = avg_three(a, b, c)


print(f"Average: {avg_pi.result():.5f}")

<P><BR>
<table style="width:90%">
    <tr><td style="text-align:left;"><a href="./3 - Configurations.ipynb">Previous (Configurations)</a></td><td style="text-align:right;"><a href="./5 - Debugging.ipynb">Next (Debugging)</a></td></tr>
</table>
